In [ ]:
!pip install transformers sentence-transformers faiss-cpu torch

In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer, util
import faiss
import torch
import os

# Set Hugging Face token from environment variable (if needed)
hf_token = "" # Replace "HF_TOKEN" with your env variable name

# Device setup
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# 1. Translation Models (Low-resource <-> English)
# Target low-resource language: Amharic (am)
low_resource_lang = "am"  # Amharic language code
high_resource_lang = "en"  # English

# Load M2M100 model once for both directions (1.2B parameters)
# translator_model_name = "facebook/m2m100_1.2B"  # Multilingual model supporting Amharic
# translator_model_name = "facebook/nllb-200-distilled-600M"
translator_model_name = "facebook/nllb-200-3.3B"  # Latest high-performance translation model

translator_tokenizer = AutoTokenizer.from_pretrained(translator_model_name, token=hf_token)
translator_model = AutoModelForSeq2SeqLM.from_pretrained(translator_model_name, token=hf_token)

# Amharic to English translation pipeline
translator_to_en = pipeline(
    "translation",
    model=translator_model,
    tokenizer=translator_tokenizer,
    src_lang="am",  # Source language: Amharic
    tgt_lang="en",  # Target language: English
    device=0 if torch.cuda.is_available() else -1  # Use GPU if available
)

# English to Amharic translation pipeline (reusing the same model)
translator_from_en = pipeline(
    "translation",
    model=translator_model,
    tokenizer=translator_tokenizer,
    src_lang="en",  # Source language: English
    tgt_lang="am",  # Target language: Amharic
    device=0 if torch.cuda.is_available() else -1  # Use GPU if available
)

# 2. Sentence Embeddings Model (for RAG)
embed_model_name = "all-mpnet-base-v2"  # Good general-purpose sentence embeddings
embed_model = SentenceTransformer(embed_model_name, device=device)  # Use GPU if available

# 3. Generative AI Model (GenAI) - Llama 2 (Requires Hugging Face Token and Access)
gen_ai_model_name = "meta-llama/Llama-2-7b-chat-hf"  # Llama 2 7B Chat

try:
    gen_ai_tokenizer = AutoTokenizer.from_pretrained(gen_ai_model_name, token=hf_token)
    gen_ai_model = AutoModelForCausalLM.from_pretrained(
        gen_ai_model_name,
        torch_dtype=torch.float16,  # Half precision for memory efficiency
        device_map="auto",  # Auto-distribute across GPU(s)
        token=hf_token
    )
    gen_ai_pipeline = pipeline(
        "text-generation",
        model=gen_ai_model,
        tokenizer=gen_ai_tokenizer,
        torch_dtype=torch.float16,
        device_map="auto",
    )
except Exception as e:
    print(f"Error loading Llama-2 model: {e}")
    gen_ai_pipeline = None  # Fallback if Llama-2 fails to load

print("Models Loaded!")

# Optional: Test the translation
if __name__ == "__main__":
    # Test Amharic to English
    text_am = "ሰላም አለም"  # "Hello World" in Amharic
    result_to_en = translator_to_en(text_am)
    print(f"Amharic to English: {result_to_en[0]['translation_text']}")

    # Test English to Amharic
    text_en = "How are you?"
    result_to_am = translator_from_en(text_en)
    print(f"English to Amharic: {result_to_am[0]['translation_text']}")

Using device: cpu


tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Error loading Llama-2 model: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Llama-2-7b-chat-hf.
403 Client Error. (Request ID: Root=1-67bfbdb4-2d67e9cf4c639b4d7e447a21;345b7584-ffcc-4951-96ed-02267bd299db)

Cannot access gated repo for url https://huggingface.co/meta-llama/Llama-2-7b-chat-hf/resolve/main/config.json.
Your request to access model meta-llama/Llama-2-7b-chat-hf has been rejected by the repo's authors.
Models Loaded!
Amharic to English: enenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenen
English to Amharic: am Howen are you? Howen are you? Howen are you? Howen are you? Howe

In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer, util
import faiss
import torch
import os

# Set Hugging Face token from environment variable (if needed)
hf_token = "" # Replace "HF_TOKEN" with your env variable name

# Device setup
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# 1. Translation Models (Low-resource <-> English)
# Target low-resource language: Amharic (am)
low_resource_lang = "am"  # Amharic language code
high_resource_lang = "en"  # English

# Load M2M100 model once for both directions (1.2B parameters)
translator_model_name = "facebook/m2m100_1.2B"  # Multilingual model supporting Amharic
# translator_model_name = "facebook/nllb-200-distilled-600M"
# translator_model_name = "facebook/nllb-200-3.3B"  # Latest high-performance translation model

translator_tokenizer = AutoTokenizer.from_pretrained(translator_model_name, token=hf_token)
translator_model = AutoModelForSeq2SeqLM.from_pretrained(translator_model_name, token=hf_token)

# Amharic to English translation pipeline
translator_to_en = pipeline(
    "translation",
    model=translator_model,
    tokenizer=translator_tokenizer,
    src_lang="am",  # Source language: Amharic
    tgt_lang="en",  # Target language: English
    device=0 if torch.cuda.is_available() else -1  # Use GPU if available
)

# English to Amharic translation pipeline (reusing the same model)
translator_from_en = pipeline(
    "translation",
    model=translator_model,
    tokenizer=translator_tokenizer,
    src_lang="en",  # Source language: English
    tgt_lang="am",  # Target language: Amharic
    device=0 if torch.cuda.is_available() else -1  # Use GPU if available
)

# 2. Sentence Embeddings Model (for RAG)
embed_model_name = "all-mpnet-base-v2"  # Good general-purpose sentence embeddings
embed_model = SentenceTransformer(embed_model_name, device=device)  # Use GPU if available

# 3. Generative AI Model (GenAI) - Llama 2 (Requires Hugging Face Token and Access)
gen_ai_model_name = "meta-llama/Llama-2-7b-chat-hf"  # Llama 2 7B Chat

try:
    gen_ai_tokenizer = AutoTokenizer.from_pretrained(gen_ai_model_name, token=hf_token)
    gen_ai_model = AutoModelForCausalLM.from_pretrained(
        gen_ai_model_name,
        torch_dtype=torch.float16,  # Half precision for memory efficiency
        device_map="auto",  # Auto-distribute across GPU(s)
        token=hf_token
    )
    gen_ai_pipeline = pipeline(
        "text-generation",
        model=gen_ai_model,
        tokenizer=gen_ai_tokenizer,
        torch_dtype=torch.float16,
        device_map="auto",
    )
except Exception as e:
    print(f"Error loading Llama-2 model: {e}")
    gen_ai_pipeline = None  # Fallback if Llama-2 fails to load

print("Models Loaded!")

# Optional: Test the translation
if __name__ == "__main__":
    # Test Amharic to English
    text_am = "ሰላም አለም"  # "Hello World" in Amharic
    result_to_en = translator_to_en(text_am)
    print(f"Amharic to English: {result_to_en[0]['translation_text']}")

    # Test English to Amharic
    text_en = "How are you?"
    result_to_am = translator_from_en(text_en)
    print(f"English to Amharic: {result_to_am[0]['translation_text']}")

Using device: cpu


tokenizer_config.json:   0%|          | 0.00/271 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/909 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/3.71M [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/233 [00:00<?, ?B/s]

Device set to use cpu
Device set to use cpu


model.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

Error loading Llama-2 model: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Llama-2-7b-chat-hf.
403 Client Error. (Request ID: Root=1-67c01df1-6025cddc10d4b82b16267800;b85edc87-5cbe-484c-82fa-e37d7642fe01)

Cannot access gated repo for url https://huggingface.co/meta-llama/Llama-2-7b-chat-hf/resolve/main/config.json.
Your request to access model meta-llama/Llama-2-7b-chat-hf has been rejected by the repo's authors.
Models Loaded!
Amharic to English: Hello world
English to Amharic: አንተ እንዴት ነው?


In [22]:
text_e = "Liverpool extended their lead over Arsenal to 13 points with a midweek Premier League win. Liverpool drew 2-0 at Newcastle on Wednesday night, while Arsenal were held to a goalless draw at Nottingham Forest. No team has ever lost the Premier League title by such a margin with 10 games remaining. The Reds have only lost once this season. The fact that title rivals Arsenal have only taken one point from their last two games has helped Liverpool's progress. Arsenal manager Mikel Arteta, for his part, said before the Nottingham game that he absolutely did not accept the idea that the trophy had been taken from Arsenal, adding that after the game it was still the same as a week or two ago. All we can do is win games, he told the BBC."
result_to_am = translator_from_en(text_e)
print(f"English to Amharic: {result_to_am[0]['translation_text']}")

English to Amharic: amasa: Liverpool drew 2-0 at Newcastle on Wednesday night, while Arsenal were held to a goalless draw at Nottingham Forest. No team has ever lost the Premier League title by such a margin with 10 games remaining. The Reds have only lost once this season. The fact that title rivals Arsenal have taken only one point from their last two games has helped Liverpool's progress. Arsenal manager Mikel Arteta, for his part, said before the Nottingham game that he absolutely did not accept the idea that the trophy had been taken from Arsenal, adding that after the game it was still the same as a week or two ago. all we can do is win games, he told the BBC.


In [20]:
text_e = "በሳምንቱ አጋማሽ የፕሪሚዬር ሊግ ጨዋታ ድል የቀናው ሊቨርፑል ከአርሰናል ጋር ያለውን የነጥብ ልዩነት ወደ 13 ማስፋት ችሏል። ረቡዕ ምሽት በሜዳው አንፊልድ ኒውካስልን ያስተናገደው ሊቨርፑል ሁለት ለምንም ሲረታ ወደ ኖቲንግሀም ሜዳ ያቀናው አርሰናል ደግሞ ያለጎል ተለያይቷል። በእንግሊዝ ፕሪሚዬር ሊግ ታሪክ ውድድሩ 10 ጨዋታዎች እየቀሩት በዚህ የነጥብ ልዩነት እየመራ ዋንጫውን ያጣ ክለብ የለም። ቀያዮቹ በያዝነው የውድድር ዘመን አንድ ጊዜ ብቻ ነው ሽንፈት የቀመሱት። የዋንጫ ተፎካካሪው አርሰናል ካለፉት ሁለት ጨዋታዎች አንድ ነጥብ ብቻ ማግኘቱ ለሊቨርፑል ግስጋሴ አስተዋፅዖ አድርጓል። የሊቨርፑሉ አሰልጣኝ አርን ስሎት አሁንም ገና ብዙ ይቀረናል። 10 ጨዋታዎች አሉ ሲሉ አስተያየታቸውን ሰጥተዋል። የአርሰናሉ አሰልጣኝ ሚኬል አርቴታ በበኩላቸው ከኖቲንግሀም ጨዋታ በፊት በሰጡት አስተያየት ዋንጫው ከአርሰናል እጅ ወጥቷል የሚለውን ሐሳብ በፍፁም አልቀበለውም ብለዋል። ከጨዋታው በኋላ ደግሞ አሁንም ከሳምንት ከሁለት ሳምንት በፊት ተመሳሳይ ነው። እኛ ማድረግ የምንችለው ጨዋታዎችን ማሸነፍ ነው ሲሉ ለቢቢሲ ተናግረዋል።"
result_to_en = translator_to_en(text_e)
print(f"English to Amharic: {result_to_en[0]['translation_text']}")

Your input_length: 312 is bigger than 0.9 * max_length: 200. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)


English to Amharic: enredredreds have lost only once in the past season. the rivalry rivalry has only lost one point in the last two games. if the rivalry rivalry has lost only one point in the last two games, we can still make a suggestion to Liverpool. we have still made suggestions to coach the team. we have still made suggestions to play before the match.


In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer, util
import faiss
import torch
import os

# Set Hugging Face token from environment variable (if needed)
hf_token = "" # Replace "HF_TOKEN" with your env variable name

# Device setup
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# 1. Translation Models (Low-resource <-> English)
# Target low-resource language: Amharic (am)
low_resource_lang = "am"  # Amharic language code
high_resource_lang = "en"  # English

# Load M2M100 model once for both directions (1.2B parameters)
# translator_model_name = "facebook/m2m100_1.2B"  # Multilingual model supporting Amharic
translator_model_name = "facebook/nllb-200-distilled-600M"
# translator_model_name = "facebook/nllb-200-3.3B"  # Latest high-performance translation model

translator_tokenizer = AutoTokenizer.from_pretrained(translator_model_name, token=hf_token)
translator_model = AutoModelForSeq2SeqLM.from_pretrained(translator_model_name, token=hf_token)

# Amharic to English translation pipeline
translator_to_en = pipeline(
    "translation",
    model=translator_model,
    tokenizer=translator_tokenizer,
    src_lang="am",  # Source language: Amharic
    tgt_lang="en",  # Target language: English
    device=0 if torch.cuda.is_available() else -1  # Use GPU if available
)

# English to Amharic translation pipeline (reusing the same model)
translator_from_en = pipeline(
    "translation",
    model=translator_model,
    tokenizer=translator_tokenizer,
    src_lang="en",  # Source language: English
    tgt_lang="am",  # Target language: Amharic
    device=0 if torch.cuda.is_available() else -1  # Use GPU if available
)

# 2. Sentence Embeddings Model (for RAG)
embed_model_name = "all-mpnet-base-v2"  # Good general-purpose sentence embeddings
embed_model = SentenceTransformer(embed_model_name, device=device)  # Use GPU if available

# 3. Generative AI Model (GenAI) - Llama 2 (Requires Hugging Face Token and Access)
gen_ai_model_name = "meta-llama/Llama-2-7b-chat-hf"  # Llama 2 7B Chat

try:
    gen_ai_tokenizer = AutoTokenizer.from_pretrained(gen_ai_model_name, token=hf_token)
    gen_ai_model = AutoModelForCausalLM.from_pretrained(
        gen_ai_model_name,
        torch_dtype=torch.float16,  # Half precision for memory efficiency
        device_map="auto",  # Auto-distribute across GPU(s)
        token=hf_token
    )
    gen_ai_pipeline = pipeline(
        "text-generation",
        model=gen_ai_model,
        tokenizer=gen_ai_tokenizer,
        torch_dtype=torch.float16,
        device_map="auto",
    )
except Exception as e:
    print(f"Error loading Llama-2 model: {e}")
    gen_ai_pipeline = None  # Fallback if Llama-2 fails to load

print("Models Loaded!")

# Optional: Test the translation
if __name__ == "__main__":
    # Test Amharic to English
    text_am = "ሰላም አለም"  # "Hello World" in Amharic
    result_to_en = translator_to_en(text_am)
    print(f"Amharic to English: {result_to_en[0]['translation_text']}")

    # Test English to Amharic
    text_en = "How are you?"
    result_to_am = translator_from_en(text_en)
    print(f"English to Amharic: {result_to_am[0]['translation_text']}")

Using device: cpu


tokenizer_config.json:   0%|          | 0.00/564 [00:01<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

Device set to use cpu
Device set to use cpu


model.safetensors:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

Error loading Llama-2 model: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Llama-2-7b-chat-hf.
403 Client Error. (Request ID: Root=1-67c04cd7-58267822318d0b021f83760a;25612ea7-cfaa-47f9-90e5-6d50631da222)

Cannot access gated repo for url https://huggingface.co/meta-llama/Llama-2-7b-chat-hf/resolve/main/config.json.
Your request to access model meta-llama/Llama-2-7b-chat-hf has been rejected by the repo's authors.
Models Loaded!
Amharic to English: en peace world am
English to Amharic: am, how are you?


In [19]:
text_e = "በሳምንቱ አጋማሽ የፕሪሚዬር ሊግ ጨዋታ ድል የቀናው ሊቨርፑል ከአርሰናል ጋር ያለውን የነጥብ ልዩነት ወደ 13 ማስፋት ችሏል። ረቡዕ ምሽት በሜዳው አንፊልድ ኒውካስልን ያስተናገደው ሊቨርፑል ሁለት ለምንም ሲረታ ወደ ኖቲንግሀም ሜዳ ያቀናው አርሰናል ደግሞ ያለጎል ተለያይቷል። በእንግሊዝ ፕሪሚዬር ሊግ ታሪክ ውድድሩ 10 ጨዋታዎች እየቀሩት በዚህ የነጥብ ልዩነት እየመራ ዋንጫውን ያጣ ክለብ የለም። ቀያዮቹ በያዝነው የውድድር ዘመን አንድ ጊዜ ብቻ ነው ሽንፈት የቀመሱት። የዋንጫ ተፎካካሪው አርሰናል ካለፉት ሁለት ጨዋታዎች አንድ ነጥብ ብቻ ማግኘቱ ለሊቨርፑል ግስጋሴ አስተዋፅዖ አድርጓል። የሊቨርፑሉ አሰልጣኝ አርን ስሎት አሁንም ገና ብዙ ይቀረናል። 10 ጨዋታዎች አሉ ሲሉ አስተያየታቸውን ሰጥተዋል። የአርሰናሉ አሰልጣኝ ሚኬል አርቴታ በበኩላቸው ከኖቲንግሀም ጨዋታ በፊት በሰጡት አስተያየት ዋንጫው ከአርሰናል እጅ ወጥቷል የሚለውን ሐሳብ በፍፁም አልቀበለውም ብለዋል። ከጨዋታው በኋላ ደግሞ አሁንም ከሳምንት ከሁለት ሳምንት በፊት ተመሳሳይ ነው። እኛ ማድረግ የምንችለው ጨዋታዎችን ማሸነፍ ነው ሲሉ ለቢቢሲ ተናግረዋል።"
result_to_am = translator_from_en(text_e)
print(f"English to Amharic: {result_to_am[0]['translation_text']}")

Your input_length: 312 is bigger than 0.9 * max_length: 200. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)


English to Amharic: am-game.com - The Red have lost only once in the past season.The Red have lost only once in the last two games.The Cup contender has lost only one point in the last two games.If we have lost only one point in the last two games, then it is still a matter of time before the team has lost the game.We have still gotten feedback from the coaches that they have lost the game.
